In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from IPython.display import display
display(tf.__version__)

'2.8.0'

In [4]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3N4BKW/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3N4BKW/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [27]:
train_sen_test = [s.numpy() for s, l in train]

In [32]:
display(train_sen_test[0].decode('utf-8'))

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [33]:
import numpy as np
train, test = imdb['train'], imdb['test']

train_sentences = [s.numpy().decode('utf-8') for s, l in train]
train_labels = np.array([l.numpy() for s, l in train])

test_sentences = [s.numpy().decode('utf-8') for s, l in test]
test_labels = np.array([l.numpy() for s, l in test])

In [34]:
for s, l in train:
  print(s)
  print(l)
  break

type(train_sentences)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


list

In [35]:
oov_token = "<OOV>"
vocab_size = 10000
pad_type = 'post'
trunc_type = 'post'
max_length = 120
embedding_dims = 16


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)


In [38]:
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen=max_length, padding=pad_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, truncating=trunc_type, maxlen=max_length, padding=pad_type)

display(train_sentences[0])
print(word_index['this'], word_index['was'], word_index['an'], word_index['absolutely'])
display(padded[0])
display(len(padded[0]))

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

12 14 33 425


array([  12,   14,   33,  425,  392,   18,   90,   28,    1,    9,   32,
       1366, 3585,   40,  486,    1,  197,   24,   85,  154,   19,   12,
        213,  329,   28,   66,  247,  215,    9,  477,   58,   66,   85,
        114,   98,   22, 5675,   12, 1322,  643,  767,   12,   18,    7,
         33,  400, 8170,  176, 2455,  416,    2,   89, 1231,  137,   69,
        146,   52,    2,    1, 7577,   69,  229,   66, 2933,   16,    1,
       2904,    1,    1, 1479, 4940,    3,   39, 3900,  117, 1584,   17,
       3585,   14,  162,   19,    4, 1231,  917, 7917,    9,    4,   18,
         13,   14, 4139,    5,   99,  145, 1214,   11,  242,  683,   13,
         48,   24,  100,   38,   12, 7181, 5515,   38, 1366,    1,   50,
        401,   11,   98, 1197,  867,  141,   10,    0,    0,    0],
      dtype=int32)

120

In [43]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Embedding(vocab_size, embedding_dims, input_length=max_length),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(6, activation='relu'),
     tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_2 (Flatten)         (None, 1920)              0         
                                                                 
 dense_4 (Dense)             (None, 6)                 11526     
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [45]:
num_epochs = 10
model.fit(padded,
          train_labels,
          epochs=num_epochs,
          validation_data = (test_padded, test_labels)
          )

Epoch 1/10
782/782 [==============================] - 11s 8ms/step - loss: 0.4834 - accuracy: 0.7535 - val_loss: 0.3783 - val_accuracy: 0.8299
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2335 - accuracy: 0.9092 - val_loss: 0.4102 - val_accuracy: 0.8202
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0902 - accuracy: 0.9762 - val_loss: 0.5101 - val_accuracy: 0.8068
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0215 - accuracy: 0.9976 - val_loss: 0.5990 - val_accuracy: 0.8070
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0052 - accuracy: 0.9997 - val_loss: 0.6771 - val_accuracy: 0.8077
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.7360 - val_accuracy: 0.8080
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 8.5378e-04 - accuracy: 1.0000 - val_loss: 0.7874 - val_accuracy: 0.8082
E